In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from datetime import time
import matplotlib.pyplot as pplot
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

/home/matheuspds/miniconda3/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence
/home/matheuspds/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [145]:
volume_df_1 = pd.read_csv("../dataset/volume(table 6)_training.csv")
volume_df_2 = pd.read_csv("../dataset/volume(table 6)_training2.csv")
volume_df_2 = volume_df_2.rename(columns = {'date_time':'time','tollgate':'tollgate_id','is_etc':'has_etc','veh_type':'vehicle_type','model':'vehicle_model'})


In [146]:
volume_df_list = [volume_df_1, volume_df_2]

In [147]:
volume_df = pd.concat(volume_df_list)

In [148]:
volume_df.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type
0,2016-09-19 23:09:25,2,0,1,0,NaN
1,2016-09-19 23:11:53,2,0,1,0,NaN
2,2016-09-19 23:13:54,2,0,1,0,NaN
3,2016-09-19 23:17:48,1,0,1,1,NaN
4,2016-09-19 23:16:07,2,0,1,0,NaN


In [131]:
volume_df = pd.read_csv("../dataset/volume(table 6)_test2.csv")
volume_df = volume_df.rename(columns = {'date_time':'time','tollgate':'tollgate_id','is_etc':'has_etc','veh_type':'vehicle_type','model':'vehicle_model'})


In [149]:
volume_df['date_time'] = pd.to_datetime(volume_df['time'], format = '%Y-%m-%d %H:%M:%S')

volume_df['t'] = volume_df['date_time'].dt.time

In [150]:
volume_df['time_window'] = volume_df['t'].apply(get_timewindow)


In [151]:
volume_df.head()

,time,tollgate_id,direction,vehicle_model,has_etc,vehicle_type,date_time,t,time_window
0,2016-09-19 23:09:25,2,0,1,0,NaN,2016-09-19 23:09:25,23:09:25,"[23:00:00,23:20:00)"
1,2016-09-19 23:11:53,2,0,1,0,NaN,2016-09-19 23:11:53,23:11:53,"[23:00:00,23:20:00)"
2,2016-09-19 23:13:54,2,0,1,0,NaN,2016-09-19 23:13:54,23:13:54,"[23:00:00,23:20:00)"
3,2016-09-19 23:17:48,1,0,1,1,NaN,2016-09-19 23:17:48,23:17:48,"[23:00:00,23:20:00)"
4,2016-09-19 23:16:07,2,0,1,0,NaN,2016-09-19 23:16:07,23:16:07,"[23:00:00,23:20:00)"


In [153]:
volume_df[(volume_df['volume'] >= 100)].count()

time           2234
tollgate_id    2234
direction      2234
time_window    2234
volume         2234
dtype: int64

In [154]:
volume_df["day"] = volume_df["time"].apply(lambda x: x.day)
volume_df["hour"] = volume_df["time"].apply(lambda x: x.hour)
volume_df["minute"] = volume_df["time"].apply(lambda x: x.minute)
volume_df["week"] = volume_df["time"].apply(lambda x: x.dayofweek)
volume_df["weekend"] = volume_df["week"].apply(lambda x: 1 if x >= 5 else 0)

In [155]:
volume_df[(volume_df['day'] == 25)].head()

,time,tollgate_id,direction,time_window,volume,day,hour,minute,week,weekend
2089,2016-09-25,1,0,"[00:00:00,00:20:00)",14,25,0,0,6,1
2090,2016-09-25,1,1,"[00:00:00,00:20:00)",98,25,0,0,6,1
2091,2016-09-25,2,0,"[00:00:00,00:20:00)",3,25,0,0,6,1
2092,2016-09-25,3,0,"[00:00:00,00:20:00)",15,25,0,0,6,1
2093,2016-09-25,3,1,"[00:00:00,00:20:00)",147,25,0,0,6,1


In [156]:
volume_df['am_pm'] = volume_df["hour"].apply(lambda x: ampm(x))

In [157]:
volume_df[(volume_df['day'] == 26)].head()

,time,tollgate_id,direction,time_window,volume,day,hour,minute,week,weekend,am_pm
2438,2016-09-26 00:00:00,1,0,"[00:00:00,00:20:00)",9,26,0,0,0,0,1
2439,2016-09-26 00:00:00,1,1,"[00:00:00,00:20:00)",113,26,0,0,0,0,1
2440,2016-09-26 00:00:00,3,0,"[00:00:00,00:20:00)",15,26,0,0,0,0,1
2441,2016-09-26 00:00:00,3,1,"[00:00:00,00:20:00)",159,26,0,0,0,0,1
2442,2016-09-26 00:20:00,1,0,"[00:20:00,00:40:00)",6,26,0,20,0,0,1


In [158]:
volume_df.to_csv("treino_agregado.csv", index=False)

In [119]:
def ampm(x):
    if (x <= 12 ):
        return 1
    return 0


In [6]:
#Função que será usada para obter a janela de tempo de 20 minutos
def get_timewindow(t):
        time_window = 20
        if t.minute < time_window:
            window = [time(t.hour, 0), time(t.hour,20)]
        elif t.minute < time_window*2:
            window = [time(t.hour, 20), time(t.hour, 40)]
        else:
            try:
                window = [time(t.hour, 40), time(t.hour + 1, 0)]
            except ValueError:
                window = [time(t.hour, 40), time(0,0,0)]
        s_window = '[' + str(window[0]) + ',' + str(window[1]) + ')'
        return s_window

def get_hour(t):
        return t.hour